Нужно, используя `Spark`, `Ray datasets` или `Dask`
- найти явку (%) по всем регионам, результат отсортировать по убыванию
- выбрать  произвольного кандидата и найти тот избирательный участок, на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек)
- найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна (учитывать %)
- посчитать дисперсию по явке для каждого региона (по УИК)
- для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат


Результаты принимаются в виде `Jupyter Notebook`

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

DATA_DIR = "./materials/tasks/cik.csv"

In [2]:
data = dd.read_csv(DATA_DIR)
data.head()

,regin,tik,uik,registered_voters,total_ballots,early_voters,tik_voters,home_voters,empty_ballots,home_ballots,...,missing_ballots,extra_ballots,baburin,grudinin,zhirinovsky,putin,sobchak,suraykin,titov,yavlinsky
0,Республика Адыгея (Адыгея),Адыгейская,УИК №1,2256,2181,0,2107,62,12,62,...,0,0,0,137,32,1977,14,0,1,5
1,Республика Адыгея (Адыгея),Адыгейская,УИК №2,2700,2633,0,2575,41,17,41,...,0,0,15,86,65,2389,13,5,6,15
2,Республика Адыгея (Адыгея),Адыгейская,УИК №3,2858,2752,0,2664,75,13,75,...,0,0,1,62,13,2645,6,3,4,0
3,Республика Адыгея (Адыгея),Адыгейская,УИК №4,2066,2034,0,1857,142,35,142,...,0,0,5,288,12,1642,21,6,2,2
4,Республика Адыгея (Адыгея),Адыгейская,УИК №5,700,714,0,676,11,27,11,...,0,0,2,44,6,624,6,0,0,2


In [3]:
def get_attendance(df):
    return (df["normal_ballots"] + df["bad_ballots"]) / df["registered_voters"]

In [4]:
# найти явку (%) по всем регионам, результат отсортировать по убыванию

data_attend = data[["regin", "registered_voters", "normal_ballots", "bad_ballots"]].groupby("regin").sum()
data_attend["attend"] = get_attendance(data_attend)
data_attend.compute().sort_values("attend", ascending=False)["attend"]

regin
Территория за пределами РФ             0.980182
Республика Тыва                        0.936254
Ямало-Ненецкий автономный округ        0.918721
Кабардино-Балкарская Республика        0.916979
Республика Северная Осетия - Алания    0.899370
                                         ...   
Забайкальский край                     0.579752
Тверская область                       0.575527
Новгородская область                   0.572796
Республика Карелия                     0.571452
Иркутская область                      0.556853
Name: attend, Length: 80, dtype: float64

In [5]:
# выбрать произвольного кандидата и найти тот избирательный участок,
# на котором он получил наибольший результат (учитывать участки, на которых проголосовало больше 300 человек)

candidate = "grudinin"
data[(data["normal_ballots"] > 300) & (data[candidate] == data[candidate].max())][["regin", "tik", "uik", candidate]].compute()

,regin,tik,uik,grudinin
52319,Московская область,Ленинская,УИК №1306,1141


In [6]:
# найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна (учитывать %)


data_tik_attend = data[["regin", "tik", "registered_voters", "normal_ballots", "bad_ballots"]].groupby(["regin", "tik"]).sum()
data_tik_attend["attend"] = get_attendance(data_tik_attend)
region_min_attend, region_max_attend = data_tik_attend.groupby("regin")["attend"].min(), data_tik_attend.groupby("regin")["attend"].max()
max_range_region = (region_max_attend - region_min_attend).idxmax()
data_tik_attend.loc[max_range_region].sort_values("attend").compute()

registered_voters  \
regin                 tik                                               
Архангельская область Верхнетоемская                            14160   
                      Архангельск, Исакогорская                 26646   
                      Онежская                                  25929   
                      Лешуконская                                6957   
                      Архангельск, Соломбальская                62929   
                      Плесецкая                                 35107   
                      Виноградовская                            13877   
                      Пинежская                                 21723   
                      Шенкурская                                12263   
                      Устьянская                                24605   
                      Холмогорская                              20056   
                      Няндомская                                23124   
                      Мезенская                                  8817   
                      Архангельск, Южная                        61909   
                      Вельская                                  43024   
                      Коношская                                 18910   
                      Котласская                                15619   
                      Красноборская                             11317   
                      Каргопольская                             14936   
                      Архангельск, Ломоносовская                53356   
                      Приморская                                24146   
                      Архангельск, Октябрьская                  66689   
                      Ленская                                   10615   
                      Новодвинская городская                    30615   
                      Коряжемская городская                     30019   
                      Северодвинская городская № 2              73632   
                      Котласская городская                      57388   
                      Вилегодская                                9187   
                      Северодвинская городская № 1              75330   
                      Мирнинская городская                      19376   
                      Новая Земля                                1957   

                                                    normal_ballots  \
regin                 tik                                            
Архангельская область Верхнетоемская                          7037   
                      Архангельск, Исакогорская              13249   
                      Онежская                               13263   
                      Лешуконская                             3588   
                      Архангельск, Соломбальская             33060   
                      Плесецкая                              18496   
                      Виноградовская                          7411   
                      Пинежская                              11673   
                      Шенкурская                              6642   
                      Устьянская                             13560   
                      Холмогорская                           11082   
                      Няндомская                             12830   
                      Мезенская                               4982   
                      Архангельск, Южная                     34925   
                      Вельская                               24370   
                      Коношская                              10812   
                      Котласская                              9010   
                      Красноборская                           6529   
                      Каргопольская                           8709   
                      Архангельск, Ломоносовская             31658   
                      Приморская                             14552   
                      Архангельс

In [7]:
# посчитать дисперсию по явке для каждого региона (по УИК)

data["attend"] = get_attendance(data)
data.groupby("regin")["attend"].var().compute()

regin
Алтайский край                     0.009840
Амурская область                   0.013917
Архангельская область              0.019866
Астраханская область               0.012645
Белгородская область               0.018929
                                     ...   
Чувашская Республика - Чувашия     0.017008
Чукотский автономный округ         0.010224
Ямало-Ненецкий автономный округ    0.008625
город Москва                       0.008085
город Санкт-Петербург              0.013723
Name: attend, Length: 80, dtype: float64

In [8]:
# для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

candidates = [
    "baburin",
    "grudinin",
    "zhirinovsky",
    "putin",
    "sobchak",
    "suraykin",
    "titov",
    "yavlinsky",
]

cand_results = round(data[candidates].divide(data["normal_ballots"], axis=0).mul(100).compute())
cand_results.apply(pd.Series.value_counts)

,baburin,grudinin,zhirinovsky,putin,sobchak,suraykin,titov,yavlinsky
0.0,41958.0,1079.0,3528.0,1.0,26684.0,40148.0,46741.0,48737.0
1.0,46339.0,1680.0,3981.0,NaN,34280.0,47310.0,38403.0,29054.0
2.0,3736.0,2338.0,4031.0,NaN,17688.0,4018.0,6571.0,8734.0
3.0,510.0,2828.0,6156.0,NaN,7434.0,765.0,866.0,3560.0
4.0,170.0,3317.0,9700.0,1.0,3438.0,245.0,175.0,1560.0
...,...,...,...,...,...,...,...,...
96.0,NaN,NaN,1.0,829.0,NaN,NaN,NaN,NaN
97.0,NaN,NaN,NaN,729.0,NaN,NaN,NaN,NaN
98.0,NaN,NaN,NaN,634.0,NaN,NaN,NaN,NaN
99.0,NaN,NaN,NaN,349.0,NaN,NaN,NaN,NaN
